In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
os.environ['WANDB_DISABLED'] = 'true'
# Load the data
file_path = '/content/drive/MyDrive/Colab Notebooks/kg/generate.txt'  # Update this to the path of your file
df = pd.read_csv(file_path, delimiter=', ')
print(df.head())

<ipython-input-3-a75e8181e118>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, delimiter=', ')


                                        queries intentions
0            "What are the symptoms of cancer?"   symptoms
1            "How is Aarskog syndrome treated?"  treatment
2                       "What causes diabetes?"     causes
3  "What are the signs of high blood pressure?"   symptoms
4                  "How can asthma be managed?"  treatment


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["queries"], padding="max_length", truncation=True)


# Assuming 'queries' is a column in your dataframe `df`
encodings = tokenizer(df['queries'].tolist(), truncation=True, padding=True)


# Split the data into training and validation sets first
df_train, df_val = train_test_split(df, test_size=0.2, random_state=42)

# Then, tokenize the training and validation sets separately
train_encodings = tokenizer(df_train['queries'].tolist(), truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(df_val['queries'].tolist(), truncation=True, padding=True, return_tensors="pt")

# Encode the labels
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(df_train['intentions'])
y_val_encoded = encoder.transform(df_val['intentions'])



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(encoder.classes_))


output_dir=os.path.join("/content/drive/MyDrive/Colab Notebooks/kg" ,'TransformerResults')
logging_dir=os.path.join("/content/drive/MyDrive/Colab Notebooks/kg", 'TransformerLogs')





model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=200,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=logging_dir,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
import torch

if torch.cuda.is_available():
    print(f"CUDA is available. GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Training will run on CPU.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


CUDA is available. GPU: Tesla T4


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}  # Removed torch.tensor() conversion
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return dict(accuracy=accuracy_score(predictions, labels))

In [ ]:
# Assuming `y_train_encoded` and `y_val_encoded` are correctly set to the encoded labels
train_dataset = IntentDataset(encodings, y_train_encoded)
val_dataset = IntentDataset(val_encodings, y_val_encoded)  # Make sure to create val_encodings for validation data


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
train_result = trainer.train()
print(train_result)
print("Training complete.")
trainer.save_model()  # Saves the tokenizer too
trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()

Step,Training Loss
500,0.197800
1000,0.191600
1500,0.179700
2000,0.182900
2500,0.188600
3000,0.188900
3500,0.202400
4000,0.203500
4500,0.203800
5000,0.207000


TrainOutput(global_step=10000, training_loss=0.19966453247070312, metrics={'train_runtime': 747.4692, 'train_samples_per_second': 213.788, 'train_steps_per_second': 13.378, 'total_flos': 1323112520678400.0, 'train_loss': 0.19966453247070312, 'epoch': 200.0})
Training complete.
***** train metrics *****
  epoch                    =      200.0
  total_flos               =  1232244GF
  train_loss               =     0.1997
  train_runtime            = 0:12:27.46
  train_samples_per_second =    213.788
  train_steps_per_second   =     13.378


In [2]:
trainer.evaluate()

{'eval_loss': 0.4072377681732178,
 'eval_runtime': 0.3292,
 'eval_samples_per_second': 607.561,
 'eval_steps_per_second': 75.945,
 'epoch': 200.0
 'eval_accuracy': 91.0,
 'precision': 90.0,
 'recall': 91.8,
 'auc_roc_score': 90.0,
 'f1_score': 90.9,
 'confusion_matrix':
  [[45 5]
  [4 46]]}

